In [17]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_clean1 = application_df.drop(["EIN", "NAME"], axis=1)
application_df_clean1.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [7]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df_clean1["APPLICATION_TYPE"].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df_clean1.APPLICATION_TYPE = application_df_clean1.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df_clean1.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df_clean1["CLASSIFICATION"].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df_clean1.CLASSIFICATION = application_df_clean1.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df_clean1.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Generate our categorical variable lists
application_cat = application_df_clean1.dtypes[application_df_clean1.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df_clean1[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\SamFarber\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Merge one-hot encoded features and drop the originals
application_df_clean2 = application_df_clean1.merge(encode_df,left_index=True, right_index=True)
application_df_merged = application_df_clean2.drop(application_cat,1)
application_df_merged.head()

C:\Users\SamFarber\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [14]:
# Split our preprocessed data into our features and target arrays
y = application_df_merged["IS_SUCCESSFUL"].values
X = application_df_merged.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

C:\Users\SamFarber\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 80

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3520      
                                                                 
 dense_4 (Dense)             (None, 80)                6480      
                                                                 
 dense_5 (Dense)             (None, 1)                 81        
                                                                 
Total params: 10,081
Trainable params: 10,081
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [23]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=int(5))

In [24]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 61664.2422 - accuracy: 0.4865
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 63612.5703 - accuracy: 0.4794
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 73763.5234 - accuracy: 0.5099
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 26924.3145 - accuracy: 0.5062
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 23351.5703 - accuracy: 0.5026
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 8766.9551 - accuracy: 0.5037
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 32342.5430 - accuracy: 0.5012
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 7545.0146 - accuracy: 0.5069
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 14190.2051 - accuracy: 0.5030
Epoch 10/100
804/804 [==============================] - 2

Epoch 81/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6914 - accuracy: 0.5320
Epoch 82/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6914 - accuracy: 0.5299
Epoch 83/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6916 - accuracy: 0.5320
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6914 - accuracy: 0.5320
Epoch 85/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6915 - accuracy: 0.5320
Epoch 86/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6914 - accuracy: 0.5320
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6914 - accuracy: 0.5299
Epoch 88/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6916 - accuracy: 0.5314
Epoch 89/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6915 - accuracy: 0.5299
Epoch 90/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6915

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3520      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=int(5))

In [27]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6925 - accuracy: 0.5246
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6920 - accuracy: 0.5241
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6921 - accuracy: 0.5281
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6923 - accuracy: 0.5266
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6919 - accuracy: 0.5295
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6920 - accuracy: 0.5271
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6920 - accuracy: 0.5272
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6922 - accuracy: 0.5297
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6918 - accuracy: 0.5296
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6916 - accura

804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5320
Epoch 82/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6913 - accuracy: 0.5310
Epoch 83/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6913 - accuracy: 0.5320
Epoch 84/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6914 - accuracy: 0.5320
Epoch 85/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6914 - accuracy: 0.5320
Epoch 86/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6914 - accuracy: 0.5320
Epoch 87/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6913 - accuracy: 0.5320
Epoch 88/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6914 - accuracy: 0.5320
Epoch 89/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6914 - accuracy: 0.5320
Epoch 90/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6913 - accuracy: 

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6909 - accuracy: 0.5335 - 472ms/epoch - 2ms/step
Loss: 0.6909356117248535, Accuracy: 0.533527672290802


In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 80)                3520      
                                                                 
 dense_10 (Dense)            (None, 1)                 81        
                                                                 
Total params: 3,601
Trainable params: 3,601
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=int(5))

In [31]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 70602.8125 - accuracy: 0.4933
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 43240.9531 - accuracy: 0.5036
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 70976.8438 - accuracy: 0.5029
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 46682.0703 - accuracy: 0.5105
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 5246.4033 - accuracy: 0.5206
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 64293.3789 - accuracy: 0.5141
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 40912.9062 - accuracy: 0.5234
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 29019.0605 - accuracy: 0.5359
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 51282.2031 - accuracy: 0.5351
Epoch 10/100
804/804 [==============================] - 

804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5320
Epoch 80/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6901 - accuracy: 0.5320
Epoch 81/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6882 - accuracy: 0.5320
Epoch 82/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6900 - accuracy: 0.5320
Epoch 83/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6907 - accuracy: 0.5320
Epoch 84/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6901 - accuracy: 0.5320
Epoch 85/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6890 - accuracy: 0.5320
Epoch 86/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6884 - accuracy: 0.5320
Epoch 87/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6902 - accuracy: 0.5320
Epoch 88/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6877 - accuracy: 

In [32]:
# Export our model to HDF5 file
nn.save("refined_application.h5")